In [1]:
# imports
import numpy as np
import pandas as pd
import os, yaml
osp = os.path
from glob import glob
from importlib import reload
import matplotlib.pyplot as plt
from pathlib import Path
import dill as pickle

import torch
nn = torch.nn
F = nn.functional
%matplotlib inline
plt.rcParams["figure.figsize"] = (10.0, 8.0)
plt.rcParams['figure.dpi'] = 300

In [ ]:
# amadeus
directories_path="/data/vision/polina/users/clintonw/code/amadeus/config/main_paths.yaml"
import sys
for m in list(sys.modules.keys()):
    if m.startswith("am."):
        del(sys.modules[m])
import am.core.main
A = am.core.main.Nestor(directories_path, module_load="full")

In [2]:
import args as args_module
from data import dataloader, transforms, adni
from analysis import analyze, inception, tables, figures
import jobs as job_mgmt
import util
import monai.transforms as mtr
rescale_clip = mtr.ScaleIntensityRangePercentiles(lower=1, upper=99, b_min=0, b_max=255, clip=True, dtype=np.uint8)
rescale_noclip = mtr.ScaleIntensity(minv=0, maxv=255, dtype=np.uint8)
ANALYSIS_DIR = osp.expanduser("~/code/sitgan/analysis")

In [32]:
reload(figures);

In [ ]:
df = tables.get_results_table()
adni_jobs = df[df["dataset"] == "2D ADNI T1"].index.values

In [ ]:
figures.save_adni_trajectories()
figures.save_mrigenie_age_extrapolations()

In [10]:
job_mgmt.delete_job("failed_raw_a_cvae")
#job_mgmt.rename_job("raw_a_cvae", "failed_raw_a_cvae")

In [ ]:
figures.save_sample_outputs_for_job("dit_m_star", slurm=True)#overwrite=True

In [ ]:
analyze.get_image_matching_metrics_for_jobs(adni_jobs, slurm=True)

In [ ]:
df = tables.get_results_table()

In [ ]:
tables.save_results_table(df)

In [ ]:
df.drop("res4G", inplace=True)

In [ ]:
ages = [dp["attributes"][0] for dp in val_datalist]
ages = np.array(ages) * age_std + age_mean
plt.hist(ages)

In [ ]:
tables.print_missing_model_types()

In [ ]:
print(df[(df["model type"] == "CVAE") & (
df["G outputs"] == "")].index.values)

In [8]:
for f in util.glob2("~/code/sitgan/analysis/traj_figs/sit_a_cvae_*"):
    os.rename(f, f.replace("sit_a_cvae_", "sit_a_cvae/"))

In [21]:
plt.rcParams["figure.figsize"] = (20.0, 16.0)
jobs = ["dit_a_star", "sit_a_star"]
subject_id = "133_S_0727"
imgs = figures.compile_trajectory_for_jobs_and_subject(jobs, subject_id)
plt.imshow(imgs); plt.axis('off');

In [25]:
out = pickle.load(open(osp.expanduser(
    "~/code/sitgan/analysis/slurm/mrig_traj/89049_submitted.pkl"), "rb"))

In [ ]:
jobs = ["dit_m_star", "sit_m_star"]
subject_id = ""
imgs = figures.compile_trajectory_for_jobs_and_subject(jobs, subject_id)
plt.imshow(imgs); plt.axis('off');

In [ ]:
analyze.collect_all_missing_job_metrics()

In [ ]:
compile_trajectory_for_jobs_and_subject

In [4]:
analyze.collect_metrics_for_jobs("st_m_star", slurm=True)

In [29]:
args = job_mgmt.get_job_args("dit_m_star")

In [ ]:
#args = args_module.args_from_file(osp.expanduser("~/code/sitgan/configs/adni.yaml"))
#dataloader.get_dataloaders(args, overwrite=True)

In [ ]:
inception.fine_tune_regressor_on_dataset("2D ADNI T1", slurm=True)

In [ ]:
args = args_module.args_from_file(osp.expanduser("~/code/sitgan/configs/sit_mrig.yaml"))
dataloaders = dataloader.get_dataloaders(args)
#dataloaders = dataloader.get_dataloaders_for_dataset("2D MRI-GENIE FLAIR")

In [ ]:
batch = analyze.get_sample_batch(job="manual00", phase="val")

In [ ]:
torch.randn(1,4,6,3).tile(5,1,1,1).shape

In [ ]:
img = batch["image"][0,0].float()

In [ ]:
val_dir = osp.join(data_root, paths["MRI-GENIE"]["3D npy subdir"])

In [ ]:
path = data_dir+"/006_AX_FLAIR_T2_400000487.nii"

In [ ]:
paths = glob(data_dir+"/*")

In [ ]:
data_dir = "/data/vision/polina/scratch/clintonw/datasets/MRI-GENIE/mrigenie_flair/nii_to_npy"
path = data_dir+"/701_STROKE_FLAIR_AX_TRA_TIR_6000_100_90_0_100000485.npy"
img = np.load(path)

In [ ]:
path = "/data/vision/polina/projects/wmh/clintonw/raw_data/dcm/21/2100009092/000_FLAIR_TRA_SENSE_0"

#### rewrite dataset.bin

In [ ]:
paths["GASROS"]["2D npy subdir"]

In [ ]:
data_dir = "/data/vision/polina/scratch/clintonw/datasets/ADNI/2d_t1"
dataset = pickle.load(open(data_dir+"/dataset.bin", "rb"))

In [ ]:
pickle.dump(new_dps, open(data_dir+"/dataset.bin", "wb"))

In [ ]:
dps = dataset["datapoints"]

In [ ]:
pathsss = glob("/data/vision/polina/scratch/clintonw/amadeus/tmp/backups/ds_instance*")

In [ ]:
ix += 1
path =pathsss[ix]
dataset = pickle.load(open(path, "rb"))
print(dataset["name"])

In [ ]:
for dp in new_dps:
    if not osp.exists(data_dir+"/"+dp["ID"]+".npy"):
        print(dp)

In [ ]:
new_dps = []
for dp in dps:
    props = dp.properties
    props.pop("time instantiated", None)
    props.pop("time last seen", None)
    props.pop("time last modified", None)
    props.pop("originating actions", None)
    #props["observations"]["age"] = props["observations"]["Age"]
    if "estimates" not in props:
        pass
    elif len(props["estimates"]) == 0:
        props.pop("estimates", None)
    else:
        print(dp["name"])
    if "pipeline history" not in props:
        pass
    elif len(props["pipeline history"]) == 0:
        props.pop("pipeline history", None)
    else:
        print(dp["name"])
    new_dp = {"ID": dp["name"], **props}
    new_dps.append(new_dp)

In [ ]:
path = osp.expanduser("~/code/sitgan/configs/mrigenie.yaml")
yaml.dump(paths, open(path, "w"))

In [ ]:
paths["full MRI-GENIE"] = osp.join(paths["MRI-GENIE dataset folder"], "full_mri-genie/ds_instance.bin")

In [ ]:
path = osp.join(paths["MRI-GENIE dataset folder"], "2d_mri-genie_flair", "ds_instance.bin")
out = pickle.load(open(path, "rb"))

In [ ]:
ds = {k:out[k] for k in ["datapoints", "variables", "root folder", "training indices", "validation indices"]}

In [ ]:
dps = []
for dp in out["datapoints"]:
    dp["path"] = out["base image path"](dp)
    dps.append({k:dp[k] for k in ["path", "SiGN ID", "Porpoise ID", "GASROS ID", "clinical site", "sequence name", "observations", "slice number", "total slices"]})

In [ ]:
ds["datapoints"] = dps

In [ ]:
with open(osp.join(paths["MRI-GENIE dataset folder"], "2d_mri-genie_flair.ds"), "wb") as f:
    pickle.dump(ds, f)

### Identify artifacts

In [ ]:
# flip
# 002_AXIAL_FLAIR_400001271, 003_FLAIR_AX_0_600057300,
# 004_AXIAL_FLAIR_400001409, 009_AX_FLAIR_T2_400001742,
# 009_AX_T2_FLAIR_PROPELLER_0_400001882,

# delete:
# 003_HUVUD_FLAIR_TRANS_0_1900000003, 003_HUVUD_FLAIR_TRANS_0_1900000007,
# 003_HUVUD_FLAIR_TRANS_0_1900000017, 003_flair_T2_TIRM_TRA_DARK_FLUID_0_1800000035,
# 003_flair_T2_TIRM_TRA_DARK_FLUID_0_1800000456,
# 003_flair_T2_TIRM_TRA_DARK_FLUID_RS_0_1800000559,
# 004_AXIAL_FLAIR_400001188, 004_FLAIR_TRA_P2_1_1600000004_7
# 004_HUVUD_FLAIR_TRANS_0_1900000011, 005_AXIAL_FLAIR_0_100000291,
# 005_AX_FLAIR_T2_0_2000000721, 005_AX_FLAIR_T2_400001858,
# 005_EP2D_TRA_DARK_FLUIDFLAIR_0_100000038, 005_HUVUD_FLAIR_TRANS_0_1900000418,
# 005_T2_FLAIR_TRA_0_2300000094, 006_AX_FLAIR_T2_400001727,
# 007_AX_FLAIR_T2_400000055, 010_FLAIR_FSE_8_AXIAL_0_600051530,
# 012_T2_FLAIR_TRA_0_2300000687, 701_FLAIR_TRA_0_600058080,

In [ ]:
paths["full MRI-GENIE"] = osp.join(paths["MRI-GENIE dataset folder"], "full_mri-genie.bin")

In [ ]:
for dp in ds["datapoints"]:
    if dp["slice number"] == 7:
        img = np.load(dp["path"])
        plt.imsave(osp.join(paths['HOME'], "downloads", "flairs",
                    osp.basename(dp["path"])[:-4])+".png", img, cmap="gray")

In [ ]:
osp.basename(dp["path"])[:-4]

In [ ]:
plt.imshow(img, cmap="gray"); plt.axis('off');

In [ ]:
img_paths = sorted(glob.glob(osp.join(paths['HOME'], "downloads/flairs/*")))
ix = -1

In [ ]:
ix-=4

In [ ]:
ix += 1
path = img_paths[ix]
img1 = plt.imread(path)
ix += 1
path = img_paths[ix]
img2 = plt.imread(path)
ix += 1
path = img_paths[ix]
img3 = plt.imread(path)
img = np.concatenate((img1, img2, img3), axis=1)
plt.imshow(img, cmap="gray"); plt.axis('off');

In [ ]:
img_paths[ix-2]

In [ ]:
plt.imshow(img1, cmap="gray"); plt.axis('off');